In [1]:
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import pickle
import cv2
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(filepath='model.h5')
encoder = pickle.loads(open(file='label_encode',mode='rb').read())
# initialize the image mean for mean subtraction along with the
# predictions queue
# mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")

Q = deque(maxlen=25)

[INFO] loading model and label binarizer...
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-07-04 11:54:12.445250: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-04 11:54:12.445534: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
# initialize the video stream, pointer to output video file, and
# frame dimensions

def predict_video(path):
	vs = cv2.VideoCapture(path)
	writer = None
	(W, H) = (None, None)
	# loop over frames from the video file stream
	while True:
		# read the next frame from the file
		(grabbed, frame) = vs.read()
		# if the frame was not grabbed, then we have reached the end
		# of the stream
		if not grabbed:
			break
		# if the frame dimensions are empty, grab them
		if W is None or H is None:
			(H, W) = frame.shape[:2]
			
		# clone the output frame, then convert it from BGR to RGB
		# ordering, resize the frame to a fixed 224x224, and then
		# perform mean subtraction

		output = frame.copy()
		frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
		frame = cv2.resize(frame, (224, 224)).astype("float32")
		# frame -= mean
		
		# make predictions on the frame and then update the predictions
		# queue
		preds = model.predict(np.expand_dims(frame, axis=0),verbose = 0)[0]
		Q.append(preds)

		# perform prediction averaging over the current history of
		# previous predictions
		results = np.array(Q).mean(axis=0)
		i = np.argmax(results)

		label = encoder.classes_[i]
		
		# draw the activity on the output frame
		text = "activity: {}".format(label)
		cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 255), 2)
		# check if the video writer is None

		if writer is None:
			# initialize our video writer
			fourcc = cv2.VideoWriter_fourcc(*"MJPG")
			writer = cv2.VideoWriter('predicted.mp4', fourcc, 30,(W, H), True)

		# write the output frame to disk
		writer.write(output)
		# show the output image
		cv2.imshow("Output", output)
		key = cv2.waitKey(1) & 0xFF
		# if the `q` key was pressed, break from the loop
		
		if key == ord("q"):
			break
	# release the file pointers
	print("[INFO] cleaning up...")
	writer.release()
	vs.release()